In [5]:
from collections import namedtuple
class A():
    def __init__(self):
        self.a = 1
        self.b = 2
    @staticmethod
    def default_params():
        return{
            "a":1,
            "b":2
        }
try1 = A()
a = A.default_params()

class Params():
    def __init__(self, rnn_type):
        if rnn_type not in ['lstm', 'bilstm', 'gru']:
            raise KeyError
        self.num_classes = 8
        self.maxlen = 50
        self.batch_size = 16
        self.epochs = 10
        self.layer = rnn_type
        self.train_data_path = 'train_data/train_data.xlsx'
        self.word2vec_path = 'word2vec/word2vec_wx'
        self.model_name = 'lstm_seven_senti'
        self.embedding_train = False

params = Params('lstm')
params.num_classes
# 'log/' + params.model_name + "_train.log"
params.model_name

'lstm_seven_senti'

In [16]:
import numpy as np

a = np.zeros((0,1))
# np.concatenate((a, [1,1,2]))
a

array([], shape=(0, 1), dtype=float64)

In [35]:
import keras.backend as K

x_batch = K.placeholder(shape=(None, 256, 80))
y_batch = K.placeholder(shape=(None, 80))
y_batch = K.expand_dims(y_batch, axis=-1)
xy_batch_dot = K.batch_dot(x_batch, y_batch, axes=[2, 1])
K.int_shape(xy_batch_dot)

# def get_R(X):
#     Y, alpha = X[0], X[1]
#     ans = K.batch_dot(Y, alpha, axes=[2,1])
#     return ans

(None, 256, 1)

In [40]:
x = K.placeholder(shape=(32, 28, 3))
y = K.placeholder(shape=(3, 4))
xy = K.dot(x, y)
xy

<tf.Tensor 'Reshape_2:0' shape=(32, 28, 4) dtype=float32>

In [89]:
params = {
    'word2vec_path' : 'wordvec/word2vec_wx',
    'model_path' : 'models/lstm_seven_senti_0519.json',
    'weight_path' : 'models/lstm_seven_senti_0519_weight.h5'
}

def load_predict_data(data_path, target_column, params, dict_w):
    comment = pd.read_excel(data_path)
    cw = lambda x: list(jieba.cut(str(x))) #定义分词函数
    maxlen = 50
    filter_word = lambda x: [a for a in x if a in dict_w.index]
    get_sent = lambda x: list(dict_w['id'][x])
    comment = comment[comment[target_column].notnull()] #仅读取非空评论
    comment['words'] = comment[target_column].apply(cw) #评论分词
    comment['words'] = comment['words'].apply(filter_word)
    comment['sent'] = comment['words'].apply(get_sent)
    comment['sent'] = list(sequence.pad_sequences(comment['sent'], maxlen=maxlen, padding='post', truncating='post'))
    return comment
a = load_predict_data('/home/jeffmxh/37_5data.xlsx', 'content', params, dict_w)

In [92]:
len(a['sent'][0])

50

In [85]:
get_sent = lambda x: list(dict_w['id'][x])
b = get_sent(['hahaha','我'])
b = a['words'][0]
b = [x for x in b if x in dict_w.index]
get_sent(b)

[140, 172, 2, 26, 12848, 7, 4, 7]

In [59]:
b = a['sent'][0]
print(b)
type(b[7])

[140.0, 172.0, 2.0, 26.0, 12848.0, 7.0, nan, nan, 4.0, 7.0, nan]


TypeError: 'str' object is not callable

In [18]:
def load_word2vec(model_path):
    try:
        wordvec_model = gensim.models.word2vec.Word2Vec.load(model_path)
        wordvec_weight = wordvec_model.wv.syn0
    except:
        wordvec_model = ''
        wordvec_weight = ''
    return wordvec_model, wordvec_weight

a,b = load_word2vec('/home/jeffmxh/word2vec_wx/word2vec_wx')
a
if a:
    print('hahaha')

hahaha


In [19]:
# from __future__ import absolute_import #导入3.x的特征函数
# from __future__ import print_function
import pandas as pd #导入Pandas
import keras
import numpy as np #导入Numpy
import jieba #导入结巴分词
jieba.enable_parallel(32)
import h5py
import gensim
# import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras import backend as K
from keras import metrics
from multiprocessing import Pool
from collections import namedtuple

'''
参数设置
'''
params = {
    'num_classes' : 8,
    'maxlen' : 50,
    'batch_size' : 16,
    'epochs' : 10,
    'layer':'lstm',
    'train_data_path' : '/home/jeffmxh/ML_learn/emotion_classify/train_data/train_data.xlsx',
    'word2vec_path' : '/home/jeffmxh/ML_learn/emotion_classify/word2vec/word2vec_wx',
    'model_path' : 'models/2gru_seven_senti_0509_train.json',
    'weight_path' : 'models/2gru_seven_senti_0509_train_weight.h5',
    'embedding_train' : False
}

Train_Set = namedtuple('Train_Set', 'x y xt yt xa ya')

def trans_emo(emo):
    trans_dict = dict(zip(['none', 'disgust', 'like', 'happiness', 'sadness', 'surprise', 'anger', 'fear'], range(8)))
    return trans_dict[emo]

def load_word2vec(model_path):
    wordvec_model = gensim.models.word2vec.Word2Vec.load(model_path)
    wordvec_weight = wordvec_model.wv.syn0
    return wordvec_model, wordvec_weight

def preprocess_data(file_path, wordvec_model, params):
    raw_data = pd.read_excel(file_path)
    print('Data loaded!')
    data = pd.DataFrame({'sent' : raw_data.sentence,
                         'mark' : raw_data.emotion_1 })
    data['mark'] = data['mark'].apply(trans_emo)
    print('emotion_tag transformed!')
    cw = lambda x: list(jieba.cut(str(x))) #定义分词函数
    data['words'] = data['sent'].apply(cw)
    vocab = dict([(k, v.index) for k, v in wordvec_model.wv.vocab.items()])
    word_to_id = lambda word: not (vocab.get(word) is None) and vocab.get(word) or 0
    words_to_ids = lambda words: list(map(word_to_id, words))
    data['sent'] = data['words'].apply(words_to_ids)
    reverse_seq = lambda id_seq: id_seq[::-1]
    concat_seq = lambda a,b: list(np.hstack((a, b)))
    print("Pad sequences (samples x time)")
    data['sent_rev'] = list(sequence.pad_sequences(data['sent'], maxlen=params['maxlen']))
    data['sent_rev'] = data['sent_rev'].apply(reverse_seq)
    data['sent'] = list(sequence.pad_sequences(data['sent'], maxlen=params['maxlen'], padding='post', truncating='post'))
    data['sent'] = data['sent'].combine(data['sent_rev'], func=concat_seq)
    return data

def split_data(train_data):
    x = np.array(list(train_data['sent']))[::2] #训练集
    y = np.array(list(train_data['mark']))[::2]
    y = keras.utils.to_categorical(y, num_classes)
    xt = np.array(list(train_data['sent']))[1::2] #测试集
    yt = np.array(list(train_data['mark']))[1::2]
    yt = keras.utils.to_categorical(yt, num_classes)
    xa = np.array(list(train_data['sent'])) #全集
    ya = np.array(list(train_data['mark']))
    return Train_Set(x, y, xt, yt, xa, ya)

def build_model(wordvec_weight, params):
    word_embedding_layer = Embedding(
        input_dim=wordvec_weight.shape[0],
        output_dim=wordvec_weight.shape[1],
        weights=[wordvec_weight],
        trainable=params['embedding_train'])
    print('Build model...')
    model = Sequential()
    model.add(word_embedding_layer)
    model.add(Dropout(0.1))
    if params['layer']=='lstm':
        model.add(LSTM(128, return_sequences = False))
    if params['layer']=='bilstm':
        model.add(Bidirectional(LSTM(128, return_sequences = False))) 
    if params['layer']=='gru':
        model.add(GRU(128, return_sequences = False))
    model.add(Dropout(0.5))
    model.add(Dense(params['num_classes']))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.mae, metrics.categorical_accuracy])
    return model
    
print('try loading pretrained word2vec model.')
wordvec_model, wordvec_weight = load_word2vec(params['word2vec_path'])
data_all = preprocess_data(params['train_data_path'], wordvec_model, params)
# train_data = split_data(data_all)
# model = build_model(wordvec_weight, params)

# model.summary()

# model.fit(train_data.x, train_data.y, batch_size=params['batch_size'], epochs=params['epochs'], validation_data=(train_data.xt, train_data.yt))

# scores = model.evaluate(train_data.xt, train_data.yt, verbose=0)
# print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
# print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))

# json_string = model.to_json()
# with open(params['model_path'], 'wt') as f:
#     f.write(json_string)
# model.save_weights(params['weight_path'])

try loading pretrained word2vec model.
Data loaded!
emotion_tag transformed!
Pad sequences (samples x time)


In [20]:
data_all

,mark,sent,words,sent_rev
0,0,"[44916, 8803, 0, 20, 6245, 0, 18, 0, 0, 0, 0, ...","[今儿, 老爸, 逮着, 我, 一顿, 狠念, ！]","[18, 0, 6245, 20, 0, 8803, 44916, 0, 0, 0, 0, ..."
1,1,"[0, 20, 356, 8553, 100, 100, 0, 0, 0, 0, 0, 0,...","[念得, 我, 各种, 烦躁, …, …]","[100, 100, 8553, 356, 20, 0, 0, 0, 0, 0, 0, 0,..."
2,1,"[0, 167, 897, 109, 839, 1297, 1, 5379, 332, 5,...","[我要, 不要, 考虑, 下, 降低, 回家, 的, 频率, 啊, , 回来, 一次, 吵...","[18, 18, 18, 36167, 704, 183, 5, 183, 303, 882..."
3,0,"[9, 897, 451, 31, 0, 0, 239970, 0, 1233, 67, 6...","[在, 考虑, 是否, 要, 回刷, 2.3, Rom, ，, 黑, /, 锁屏, 状态, ...","[55, 2262, 7258, 4768, 0, 772, 63987, 67, 1233..."
4,0,"[0, 39643, 453, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[发烫, 耗电, 快, …]","[100, 453, 39643, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
5,1,"[37, 48, 20, 3461, 3364, 18, 0, 0, 0, 0, 0, 0,...","[这, 让, 我, 蛋, 疼, ！]","[18, 3364, 3461, 20, 48, 37, 0, 0, 0, 0, 0, 0,..."
6,2,"[511, 0, 233, 0, 11296, 1, 7819, 18, 0, 0, 0, ...","[不过, ，, 喜欢, 4.0, 原生态, 的, 界面, ！]","[18, 7819, 1, 11296, 0, 233, 0, 511, 0, 0, 0, ..."
7,0,"[1867, 1, 335, 20, 84, 33, 7, 707, 21451, 0, 1...","[去年, 的, 今天, 我, 去, 上, 了, 政治, 辅导班, ，, 回来, 看, 芒果,...","[3, 9115, 7, 25531, 954, 9, 67355, 7, 9193, 37..."
8,2,"[108, 31, 2817, 20, 1134, 0, 1, 14370, 7, 0, 2...","[又, 要, 重复, 我, 每年, 必说, 的, 台词, 了, ，, 2010, 就要, 过...","[3, 149, 9113, 64, 20, 0, 7, 710, 918, 2882, 0..."
9,2,"[1375, 0, 149, 4827, 64, 326, 3, 0, 0, 0, 0, 0...","[至少, ，, 它, 看上去, 很, 美, 。]","[3, 326, 64, 4827, 149, 0, 1375, 0, 0, 0, 0, 0..."


In [28]:
from multiprocessing import Pool
# w = [] #将所有词语整合在一起
# for i in data_all['words']:
#     w.extend(i)
dict_w = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数
dict_w['id']=list(range(1,len(dict_w)+1))
dict_w
pool = Pool(16)
get_sent = lambda x: list(dict_w['id'][x])
data['sent'] = data['words'].apply(get_sent) #速度太慢
def get_sent(x):
    return list(dict_w['id'][x])
data_all['sent'] = pool.map(get_sent, data_all['words'])

In [38]:
dict_w

,0,id
，,20362,1
的,13430,2
。,9686,3
！,5196,4
了,5082,5
我,4327,6
,3370,7
是,3209,8
你,2567,9
在,2393,10


In [31]:
import errno
import os
print(FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), 'foobar'))

[Errno 2] No such file or directory: 'foobar'


In [14]:
train_data.y

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])